In [39]:
import pandas as pd

data = pd.read_csv("CP_count_PUMA.csv")
data

,smiles,1_1,10_19,10_20,100_277,100_278,104_291,105_355,105_369,105_361,...,Cells_Neighbors_FirstClosestObjectNumber_Adjacent,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number
0,CCOC(=O)c1ccc(NC(=S)N2CCSC2c2ccc(OC)cc2)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.765625,0.890625,0.890625,1.328125,1.328125,1.328125,1.328125,0.960938,1.320312,1.328125
1,O=C(CC1NC(=O)NC1=O)Nc1cccc2ccccc12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.078125,2.718750,2.718750,2.503906,2.503906,2.503906,2.503906,3.242188,2.578125,2.503906
2,CC(Nc1nc(nc2ccccc12)N1CCCC1)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.765625,-8.140625,-8.140625,-8.250000,-8.250000,-8.250000,-8.250000,-7.710938,-7.546875,-8.250000
3,CCCC(Oc1ccc(Br)cc1)c1nc2c3cc(OC)c(OC)cc3nc(S)n2n1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.320312,-7.828125,-7.828125,-7.796875,-7.804688,-7.796875,-7.796875,-7.859375,-7.695312,-7.804688
4,CC1=C(C(NC(=O)N1)c1ccc(F)cc1)C(=O)OCc1ccc2OCOc2c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.953125,-10.390625,-10.390625,-10.960938,-10.960938,-10.960938,-10.960938,-11.406250,-10.781250,-10.960938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16165,COC(=O)[C@H]1[C@H](CO)[C@H]2Cn3c(=O)c(\C=C\C)c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.039062,-2.031250,-2.031250,-1.550781,-1.550781,-1.550781,-1.550781,-1.546875,-2.945312,-1.550781
16166,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@H](N(C)[C@@H]...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.851562,-0.359375,-0.359375,-0.835938,-0.835938,-0.835938,-0.835938,-1.328125,-0.804688,-0.835938
16167,C\C=C\c1ccc2n(C[C@@H]3[C@@H](CO)[C@@H](N(C)[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.851562,-3.359375,-3.359375,-3.835938,-3.835938,-3.835938,-3.835938,-3.328125,-4.054688,-3.835938
16168,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@@H](C(=O)N[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.039062,-2.460938,-2.460938,-3.300781,-3.300781,-3.300781,-3.300781,-3.382812,-2.937500,-3.300781


In [40]:
assays_list = data.columns.to_list()[1:-13]
len(assays_list)

270

In [41]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from tqdm import tqdm


In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier


def load_and_preprocess_data(fold):
    train_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_train_scaff.csv"
    test_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_test_scaff.csv"

    train_data = pd.read_csv(train_path)[["smiles"]]
    train_data = pd.merge(train_data, data, on="smiles")
    
    test_data = pd.read_csv(test_path)[["smiles"]]
    test_data = pd.merge(test_data, data, on="smiles")
    
    return train_data, test_data



In [62]:
from sklearn.tree import DecisionTreeClassifier, export_text
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
from tqdm import tqdm
from sklearn.utils import class_weight

# Initialize a list to store AUC scores for all folds
all_auc_scores = []

# Initialize an empty list to hold models for each fold
models_per_fold = []

# Define the fold indices (adjust based on your data split)
folds = ["0", "1", "2", "3", "4"]

# Loop through each fold, leaving it out as the test set and training on the others
for fold in tqdm(folds):
    
    train_data, _ = load_and_preprocess_data(fold)
        
    # Initialize lists to store data for the current fold's training set
    X_train_combined = pd.DataFrame()
    y_train_combined = pd.Series()
    
    # Loop through all assays and collect data
    for assay in assays_list:
        train_data_assay = train_data[~train_data[assay].isna()][[assay, "Cells_Number_Object_Number"]]
            
        # Append to combined training data
        X_train_combined = pd.concat([X_train_combined, train_data_assay[["Cells_Number_Object_Number"]]], axis=0)
        y_train_combined = pd.concat([y_train_combined, train_data_assay[assay].astype(int)], axis=0)
            

    print(len(X_train_combined))
    print(len(y_train_combined))

    
    # Initialize Decision Tree model
    clf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5, n_jobs=-1)
    
    # Train the Decision Tree on the combined data from other folds
    clf.fit(X_train_combined, y_train_combined)
       
    # Store the trained model
    models_per_fold.append(clf)

    # Access the first tree in the forest
    first_tree = clf.estimators_[0]

    # Extract the cut-off value (split point) for "Cells_Number_Object_Number" from the first tree
    cutoff_value = first_tree.tree_.threshold[0]  # The first node usually holds the main split

    # Print the cut-off value
    print(f"Cut-off value for 'Cells_Number_Object_Number' in fold {fold}: {cutoff_value}\n")


from sklearn.metrics import roc_auc_score, average_precision_score

# Predict for each individual fold using the corresponding model
for idx, fold in tqdm(enumerate(folds)):
    
    # Load and preprocess data for the current test fold
    _, test_data = load_and_preprocess_data(fold)
    
    # Get the trained model for this fold (trained on other folds)
    clf = models_per_fold[idx]
    
    # Loop through each assay in the test set for the current fold
    for assay in tqdm(assays_list):
        test_data_assay = test_data[~test_data[assay].isna()][[assay, "Cells_Number_Object_Number"]]
        
        # Separate features and target variable
        X_test = test_data_assay[["Cells_Number_Object_Number"]]
        y_test = test_data_assay[assay].astype(int)

        if len(y_test) == 0:
            print(f"Skipping assay {assay} in fold {fold} due to no test data")
            continue
        
        # Calculate number of actives and inactives in the test set
        num_actives = sum(y_test)
        num_inactives = len(y_test) - num_actives
        test_set_length = len(y_test)
        
        # Make predictions using the model for the current fold
        y_pred = clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)[:, 1]

        # Calculate AUC-ROC and AUC-PR
        auc_score = np.nan
        aucpr_score = np.nan
        try:
            auc_score = roc_auc_score(y_test, y_pred_proba)
            aucpr_score = average_precision_score(y_test, y_pred_proba)
        except:
            pass  # If there is an error, leave as NaN
       
        # Store the metrics along with fold, assay, and test set information
        all_auc_scores.append({
            "Fold": fold, 
            "Task": assay, 
            "AUC_ROC": auc_score,
            "AUC_PR": aucpr_score,
            "Test_Set_Length": test_set_length,
            "Num_Actives": num_actives,
            "Num_Inactives": num_inactives
        })

# Convert the AUC results to a DataFrame for convenience
auc_df = pd.DataFrame(all_auc_scores)

# Save the DataFrame to a CSV file
auc_df.to_csv("RF_auc_scores_across_folds.csv", index=False)

# Display the AUC DataFrame
auc_df


  0%|          | 0/5 [00:00<?, ?it/s]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


466484
466484


 20%|██        | 1/5 [00:03<00:12,  3.03s/it]

Cut-off value for 'Cells_Number_Object_Number' in fold 0: -29.25304126739502



/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


...

470810
470810


100%|██████████| 5/5 [00:15<00:00,  3.01s/it]


Cut-off value for 'Cells_Number_Object_Number' in fold 4: -26.16796875



0it [00:00, ?it/s]
100%|██████████| 270/270 [01:05<00:00,  4.10it/s]
1it [01:06, 66.18s/it]
100%|██████████| 270/270 [01:05<00:00,  4.11it/s]
2it [02:12, 66.06s/it]
100%|██████████| 270/270 [01:05<00:00,  4.09it/s]
3it [03:18, 66.15s/it]
100%|██████████| 270/270 [01:06<00:00,  4.09it/s]
4it [04:24, 66.23s/it]
100%|██████████| 270/270 [01:06<00:00,  4.07it/s]
5it [05:31, 66.28s/it]


,Fold,Task,AUC_ROC,AUC_PR,Test_Set_Length,Num_Actives,Num_Inactives
0,0,1_1,0.642857,0.111111,22,1,21
1,0,10_19,NaN,NaN,17,0,17
2,0,10_20,NaN,NaN,17,0,17
3,0,100_277,0.687500,0.250000,9,1,8
4,0,100_278,0.607143,0.325000,9,2,7
...,...,...,...,...,...,...,...
1345,4,9_18,0.400000,0.325000,7,2,5
1346,4,90_256,0.142857,0.040000,29,1,28
1347,4,91_257,1.000000,1.000000,5,1,4
1348,4,92_258,1.000000,1.000000,5,1,4


In [70]:
# Save the DataFrame to a CSV file
import pandas as pd
auc_df = pd.read_csv("RF_auc_scores_across_folds.csv")
auc_df

,Fold,Task,AUC_ROC,AUC_PR,Test_Set_Length,Num_Actives,Num_Inactives
0,0,1_1,0.642857,0.111111,22,1,21
1,0,10_19,NaN,NaN,17,0,17
2,0,10_20,NaN,NaN,17,0,17
3,0,100_277,0.687500,0.250000,9,1,8
4,0,100_278,0.607143,0.325000,9,2,7
...,...,...,...,...,...,...,...
1345,4,9_18,0.400000,0.325000,7,2,5
1346,4,90_256,0.142857,0.040000,29,1,28
1347,4,91_257,1.000000,1.000000,5,1,4
1348,4,92_258,1.000000,1.000000,5,1,4


In [71]:
len(auc_df[auc_df["Num_Actives"]==0])

338

In [72]:
mean_auc = auc_df.groupby("Task").mean().reset_index(drop=False)
mean_auc

,Task,Fold,AUC_ROC,AUC_PR,Test_Set_Length,Num_Actives,Num_Inactives
0,100_277,2.0,0.814167,0.667778,8.6,1.8,6.8
1,100_278,2.0,0.725397,0.731667,9.0,3.2,5.8
2,104_289,2.0,0.825641,0.687500,10.6,2.0,8.6
3,104_291,2.0,0.867308,0.812500,10.6,2.0,8.6
4,104_295,2.0,0.857143,0.500000,7.0,0.2,6.8
...,...,...,...,...,...,...,...
265,92_258,2.0,0.782738,0.756667,6.4,2.2,4.2
266,93_259,2.0,0.546571,0.421350,25.4,6.8,18.6
267,9_14,2.0,0.500000,0.500000,4.4,0.2,4.2
268,9_17,2.0,1.000000,1.000000,5.8,0.2,5.6


In [73]:
len(mean_auc[mean_auc["AUC_ROC"]<0.30])

12

In [74]:
len(mean_auc[mean_auc["AUC_ROC"]==1])

6

In [75]:
len(mean_auc[mean_auc["AUC_ROC"]>0.90])

19

In [76]:
len(mean_auc[mean_auc["AUC_ROC"]>0.80])

51

In [77]:
mean_auc.to_csv("RF_scaffold_mean_AUCSeal.csv")

In [78]:
mean_auc.mean()

Task                      inf
Fold                 2.000000
AUC_ROC              0.625013
AUC_PR               0.318635
Test_Set_Length    433.658519
Num_Actives         11.799259
Num_Inactives      421.859259
dtype: float64

In [79]:
mean_auc.std()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


Fold                 0.000000
AUC_ROC              0.187692
AUC_PR               0.267381
Test_Set_Length    845.995821
Num_Actives         24.261362
Num_Inactives      827.987915
dtype: float64

In [80]:
median_auc = auc_df.groupby("Task").median().reset_index(drop=False)
median_auc.to_csv("RF_scaffold_median_AUCSeal.csv")

In [85]:
median_auc

,Unnamed: 0,Task,Fold,AUC_ROC,AUC_PR,Test_Set_Length,Num_Actives,Num_Inactives
0,0,100_277,2.0,0.800000,0.722222,9.0,1.0,8.0
1,1,100_278,2.0,0.833333,0.916667,9.0,3.0,7.0
2,2,104_289,2.0,0.801282,0.625000,9.0,2.0,6.0
3,3,104_291,2.0,0.884615,0.875000,9.0,2.0,6.0
4,4,104_295,2.0,0.857143,0.500000,8.0,0.0,7.0
...,...,...,...,...,...,...,...,...
265,265,92_258,2.0,0.809524,0.916667,5.0,2.0,4.0
266,266,93_259,2.0,0.562500,0.366396,26.0,6.0,20.0
267,267,9_14,2.0,0.500000,0.500000,4.0,0.0,4.0
268,268,9_17,2.0,1.000000,1.000000,6.0,0.0,6.0


In [81]:
import pandas as pd

median_auc = pd.read_csv("RF_scaffold_median_AUCSeal.csv")

len(median_auc[median_auc["AUC_ROC"]==1])

11

In [82]:
len(median_auc[median_auc["AUC_ROC"]>0.90])

25

In [83]:
len(median_auc[median_auc["AUC_ROC"]>0.80])

56

In [84]:
len(median_auc[median_auc["AUC_ROC"]>0.70])

96